# Chains in LangChain

## Objective
* You can use multiple prompts to ingest the input variale step by step to perform different actions. With only one prompt focus on one action at one time, target result will be generated more accurately and with higher consistency, compared to one prompt generating information in multiple aspects, due to NLP attention mechanism.
* You also can choose different prompts for next step based on the analyzed resuts of previous prompts. Say analyzing Black Hole by a physics-related prompt and analyzing history by a history-related prompt, with a classification prompt to classify the subject of the input vairables

## Outline

* LLMChain
* Sequential Chains
* Router Chain

In [2]:
!pip install -q python-dotenv
!pip install -q openai==0.28.1
!pip install -q --upgrade langchain

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Get OpenAI API key
try:
    _ = load_dotenv(find_dotenv()) # read local .env file
    openai.api_key = os.environ['OPENAI_API_KEY']
except:
    openai.api_key = 'your openai key'

# Get OpenAI Model type
llm_model = "gpt-3.5-turbo"

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## LLMChain

Recall for using input variables instead of text concatentation as input prompt

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"
chain.run(product)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated

'Royal Comfort Bedding'

## SequentialChain

Provided with input for the first chain, the first chain will output vairables and we can define the name of that.

For a chain with multiple sub-chains, the variables output will be stored which can be used in later chain, e.g.
- chain_two output "summary" variable, which is used in chain_four

Finally, the SequentialChain can output with JSON format for all input & output variables.

In [6]:
from langchain.chains import SequentialChain

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)


In [12]:
review = '''
"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
'''
output = overall_chain(review)
output



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\n"Je trouve le goût médiocre. La mousse ne tient pas, c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"\n',
 'English_Review': '"I find the taste mediocre. The foam doesn\'t last, it\'s strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'summary': "The reviewer is disappointed in the taste of the product, noting that the foam doesn't last and suspects that it may be an old batch or counterfeit.",
 'followup_message': "Merci de nous avoir fait part de votre déception concernant le goût du produit. Votre avis est très important pour nous. Nous vous assurons que nous prendrons des mesures pour enquêter sur la qualité du lot en question et nous assurer que nos produits sont authentiques. Votre satisfaction est notre priorité et nous espérons pouvoir regagner votre confiance à l'avenir. Merci de votre fidélité."}

In [13]:
type(output)

dict

## Router Chain

Provided with 
- classification prompt to classify the subject of the input variable in step 1
- multiple prompts to be chosen in step 2 based on result of step 1

input for the first chain, the first chain will output vairables and we can define the name of that.

For a chain with multiple sub-chains, the variables output will be stored which can be used in later chain, e.g.
- chain_two output "summary" variable, which is used in chain_four

Finally, the SequentialChain can output with JSON format for all input & output variables.

In [25]:
# multiple prompts to be chosen
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [26]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [27]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [28]:
# generate destination_chains and destinations_str
llm = ChatOpenAI(temperature=0, model=llm_model)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [29]:
# generate a default chain if provided prompts in step 2 are not appropriate to be used
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [30]:
# classifcation prompt
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
# input "destinations_str" as "destinations" under << CANDIDATE PROMPTS >>
# to let LLM know the description of LLM chain capability and 
# output the name of the prompt
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [19]:
# Router_chain will output 
#   1. name of the prompt, and
#   2. modified version of the original input
# MultiPromptChain based on the Router_chain output (classification prompt), choose the optimal key-value pair 
# among destination_chains choies (i.e. physics, math, history, computer science)
# if not belongs to provided choices, it output "DEFAULT" and use default_chain
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [20]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [43]:
## to undermore more what the classifiation chain return

llm = ChatOpenAI(temperature=0.9, model=llm_model)

chain = LLMChain(llm=llm, prompt=router_prompt)
product = "What is black body radiation?"
chain.run(product)

'```json\n{\n    "destination": "physics",\n    "next_inputs": "What is black body radiation?"\n}\n```'

In [22]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'

In [45]:
chain.run("Why does every cell in our body contain DNA?")

'```json\n{\n    "destination": "biology",\n    "next_inputs": "Why does every cell in our body contain DNA?"\n}\n```'

# Summary

## 1. LLMChain

In the context of text manipulation and integration with libraries, LangChain adopt a different approach which are better in terms of scalability.

### Prompt praser Approach
ChatGPT primarily relies on f-strings for text manipulation, which can become cumbersome, especially in complex scenarios involving multiple variables. For example:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

```python
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)
```

### LLMChain Approach
LangChain simplifies scalability by directly interpreting variables stored within its library, eliminating the need for f-strings. This approach enhances code readability and maintainability, particularly in large-scale projects.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Similar to prompt praser in L1, LLMChain can directly query the variable

```python
LLMChain:
product = "Queen Size Sheet Set"
LLMChain(llm=llm, prompt=prompt).run(product)
```


## 2. SequentialChain

We can chain the LLMChain step by step. We can define the output vairable and stored for future chains. Finally output variables in JSON format.

Avoiding storing each output from ChatGPT, SequentialChain provide a more lean solution which you can manipulate the input and output from all processing depends on others.
(Airflow of chatgpt pipeline)

With single chain performing single action at a time, due to Attention mechanism for NLP Attention model (e.g.LLM), the accuracy and consistency of LLM will be higher.


```python
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

overall_chain(review)
```

## Conclusion
In conclusion, while both ChatGPT and LangChain serve text manipulation purposes, LangChain's innovative approach to scalability and output consistency, particularly its direct interpretation of variables and structured output handling, positions it as a preferred choice for large-scale projects and maintaining pipeline integrity. By addressing challenges such as inconsistent JSON formatting and key misalignment, LangChain offers a robust solution for seamless integration within text processing pipelines.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx